<a href="https://colab.research.google.com/github/cras-lab/Finance1/blob/main/Random_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

랜덤 그래프를 예측해 보기로 한다.
먼저 필요한 모듈을 임포트한다. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

랜덤 숫자를 300개 발생시킨다.
Seed는 256을 사용하여, 매번 동일한 랜덤이 발생하도록 한다.

In [2]:
number = 300

np.random.seed(256)
df= np.random.rand(number, 1)

sklearn에 있는 train_test_split을 사용하여, 훈련과 테스트 집합을 분할한다. 
훈련집합 크기를 지정하려면 'test_size' 인수를 사용하면 되는데, 기본값은 0.25로 설정된다. 즉 기본으로 (1-0.25) 즉 75%가 훈련집합에 할당되고, 나머지 25%가 테스트 집합에 할당된다.

In [3]:
y_train, y_test = train_test_split( df, shuffle=False  )

y값은 랜덤으로 생성했으니, X 값은 일련번호로 할당한 다음, 형태를 가로 벡터에서 세로 벡터로 변환한다.

In [4]:
X_train = np.arange(0, len(y_train) )
X_test = np.arange( len(y_train), len(df))

n_features = 1

#형태를 가로에서 세로벡터로 변환
train_series = y_train.reshape( (len(y_train), n_features))
test_series = y_test.reshape( (len(y_test), n_features))

look-back은 20으로 설정하고, KERAS에서 시계열 처리를 위한 데이터 구조를 불러와서 해당 형태로 변환한다.

In [5]:
look_back = 20

from keras.preprocessing.sequence import TimeseriesGenerator

train_generator = TimeseriesGenerator( train_series, train_series,
                                      length=look_back,
                                      batch_size = 10)


test_generator = TimeseriesGenerator(test_series, test_series,
                                     length= look_back,
                                     batch_size=10)

그래프를 도식화 하여, 어떤 모양인지 확인해 본다. 훈련과 테스트는 다른 색으로 표시한다.

In [ ]:
fig, ax = plt.subplots( 1,1, figsize=(15,4))
ax.plot( X_train, y_train, lw=3, label="train data")
ax.plot(X_test, y_test, lw=3, label="test data")
ax.legend( loc="lower left")
plt.show()

LSTM 임포트 하기

In [7]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

신경망 설계

In [9]:
n_neurons = 64 # 뉴런 개수
n_features = 1
model = Sequential()
model.add( LSTM( n_neurons, input_shape=(look_back, n_features)))
model.add( Dense(1)) #최종 출력은 1 즉 일변량 시계열 예측이다.
model.compile( optimizer='adam', loss='mse')

적합화 한 다음 예측을 수행한다.

In [10]:
model.fit( train_generator, epochs=50, verbose=0)

test_prediction = model.predict( test_generator )

6/6 [==============================] - 0s 5ms/step


결과를 그래프로 도식화 한다. 예측은 점으로 찍어 실제 테스트와 비교

In [ ]:
x= np.arange( len(df) - len(test_prediction ), len(df) )
fig, ax= plt.subplots( 1,1, figsize=(15,5))
ax.plot( X_test, y_test, lw=3, c='y', label='test_data')
ax.plot( x, test_prediction,lw=3, c='r', linestyle=':', label='predictions')
ax.legend( loc='lower left')
plt.show()